# Generate Fix Commit Tree

In [1]:
import pandas as pd

In [2]:
df = pd.read_json('../dataset/sstubsLarge')

In [3]:
print('Rows: {}, Columns: {}'.format(*df.shape))

Rows: 63923, Columns: 14


In [4]:
print('**Columns:**')
print(*df.columns, sep='\n')

**Columns:**
bugType
fixCommitSHA1
fixCommitParentSHA1
bugFilePath
fixPatch
projectName
bugLineNum
bugNodeStartChar
bugNodeLength
fixLineNum
fixNodeStartChar
fixNodeLength
sourceBeforeFix
sourceAfterFix


In [5]:
cnt = 0
def is_consecutive_fix(sf):
    global cnt
    cnt += 1
    return sf.fixCommitSHA1.isin(sf.fixCommitParentSHA1).values.sum() > 0

groups = df.groupby(['projectName', 'bugFilePath', 'bugLineNum'])
print(f'{groups.ngroups} groups')
groups = groups.filter(is_consecutive_fix)
print(f'{cnt} groups filtered')

40954 groups
40954 groups filtered


In [6]:
groups.shape

(1076, 14)

In [9]:
import sqlite3
conn = sqlite3.connect('../database/sstubs.db')
cursor = conn.cursor()

In [12]:
query = '''SELECT count(*)
    FROM sstubs_large AS b1
        INNER JOIN sstubs_large AS b2
        ON b1.child = b2.parent AND b1.project = b2.project AND b1.file = b2.file AND b1.line = b2.line'''
for res in cursor.execute(query):
    print(*res)

562


In [13]:
query = '''SELECT count(*) FROM sstubs_large WHERE (child, project, file, line) IN (
    SELECT parent, project, file, line FROM sstubs_large
)'''
for row in cursor.execute(query):
    print(*row)

399
